## Лабораторная работа №6. Классификация текста.
### Задание.
***
Для произвольного набора данных, предназначенного для классификации текстов, решите задачу классификации текста двумя способами:

Способ 1. На основе CountVectorizer или TfidfVectorizer.  
Способ 2. На основе моделей word2vec или Glove или fastText.  
Сравните качество полученных моделей.  

### Выполнение задания.
***
Задания буду выполнять на [датасете](https://github.com/bhargaviparanjape/clickbait/tree/master/dataset) из статьи "Stop Clickbait: Detecting and Preventing Clickbaits in Online News Media".  
Датасет предполагает бинарную классификацию.

In [47]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from scipy import stats
from IPython.display import Image
from sklearn.datasets import load_iris, load_boston
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import SVC, NuSVC, LinearSVC, OneClassSVM, SVR, NuSVR, LinearSVR
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")

Датасет состоит из 2х файлов (кликбейт и не кликбейт заголовки). Загрузим их, установим значение целевого признак, объеденим датасеты в единый DataFrame и перемешаем строки:

In [20]:
data_clickbait = pd.read_csv("../data/clickbait_data.txt", sep = "\n\n", engine="python", names = ["header_text"], header = None)
data_clickbait["is_clickbait"] = [1] * data_clickbait.shape[0]
data_non_clickbait = pd.read_csv("../data/non_clickbait_data.txt", sep = "\n\n", engine="python", names = ["header_text"], header = None)
data_non_clickbait["is_clickbait"] = [0] * data_non_clickbait.shape[0]
dataset = data_clickbait.append(data_non_clickbait)
dataset = dataset.sample(frac = 1, random_state = 100)
dataset = dataset.reset_index(drop = True)
dataset

,header_text,is_clickbait
0,"In Michigan, Bank Lends Little of Its Bailout ...",0
1,"Four dead, more than a million in U.S. without...",0
2,"In Wyoming, Debate Rages Over Elk Feeding Program",0
3,Bryant and Lakers Return to the N.B.A. Finals,0
4,This Baby's Reaction To Hearing About How She ...,1
...,...,...
31995,"When You Binge-Watch ""Making A Murderer"" And T...",1
31996,Schiphol airliner crash blamed on altimeter fa...,0
31997,Radiohead Release Rejected Bond Theme Song And...,1
31998,Chernobyl Taking a Toll on Invertebrates Too,0


In [36]:
documents = dataset["header_text"].tolist()

### Векторизация на основе TfidfVectorizer

In [38]:
tfidf_vectorizer = TfidfVectorizer()
vocabulary = tfidf_vectorizer.fit(documents) # Формируем набор признаков и параметры idf
print("Число признаков: {}".format(len(vocabulary.vocabulary_)))
documents_vectorized = tfidf_vectorizer.transform(documents)

Число признаков: 22761


In [45]:
print("Первый документ, векторизованный (не нулевые признаки): {}"
      .format(str([i for i in documents_vectorized.todense()[0].getA1() if i > 0])))
print("Первый документ, оригинальный: {}".format(documents[0]))

Первый документ, векторизованный (не нулевые признаки): [0.38686554025791037, 0.3186643872558275, 0.395177358289709, 0.1288209792126749, 0.2891144884412702, 0.4776795401471119, 0.31294434961282225, 0.38248152507739863, 0.14155161385070417]
Первый документ, оригинальный: In Michigan, Bank Lends Little of Its Bailout Funds
